In [348]:
from pathlib import Path
from dataclasses import dataclass, field
import string
from typing import List
from copy import deepcopy

import sys
MAXINT = sys.maxsize -1

In [407]:
@dataclass
class Robots:
    ore : int = 1
    clay : int = 0
    obsidian : int = 0
    geode : int = 0

    def increment(self, s : str):
        match s:
            case 'ore':
                self.ore +=1
            case 'clay':
                self.clay +=1
            case 'obsidian':
                self.obsidian +=1
            case 'geode':
                self.geode +=1

    @property
    def _as_tuple(self):
        return (self.ore, self.clay, self.obsidian, self.geode)

    def __iter__(self):
        return iter(self._as_tuple)
    def __hash__(self):
        return hash(self._as_tuple)



@dataclass
class Resources:
    ore : int = 0
    clay : int = 0
    obsidian : int = 0
    geode : int = 0

    @property
    def _as_tuple(self):
        return (self.ore, self.clay, self.obsidian, self.geode)

    def __iter__(self):
        return iter(self._as_tuple)
    def __hash__(self):
        return hash(self._as_tuple)

    def can_afford(self, other):
        return (self.ore >= other.ore) and \
               (self.clay >= other.clay) and \
               (self.obsidian >= other.obsidian) and \
               (self.geode >= other.geode)

    def pay(self, other):
        if self.ore != MAXINT:
            self.ore -= other.ore
        if self.clay != MAXINT:
            self.clay -= other.clay
        if self.obsidian != MAXINT:
            self.obsidian -= other.obsidian
        if self.geode != MAXINT:
            self.geode -= other.geode


@dataclass
class Blueprint:
    ore : Resources
    clay : Resources
    obsidian : Resources
    geode : Resources

    def max_costs(self):
        max_ore = max(self.ore.ore, self.clay.ore, self.obsidian.ore, self.geode.ore)
        max_clay = max(self.ore.clay, self.clay.clay, self.obsidian.clay, self.geode.clay)
        max_obsidian = max(self.ore.obsidian, self.clay.obsidian, self.obsidian.obsidian, self.geode.obsidian)
        max_geode = max(self.ore.geode, self.clay.geode, self.obsidian.geode, self.geode.geode)

        return Resources(max_ore, max_clay, max_obsidian, max_geode)


@dataclass
class State:
    robots : Robots = field(default_factory=Robots)
    resources : Resources = field(default_factory=Resources)

    def mine(self):
        self.resources.ore += self.robots.ore
        self.resources.clay += self.robots.clay
        self.resources.obsidian += self.robots.obsidian
        self.resources.geode += self.robots.geode

    def set_inf(self, max_costs):
        if min(self.robots.ore,self.resources.ore) >= max_costs.ore:
            self.resources.ore = MAXINT
        if min(self.robots.clay, self.resources.clay) >= max_costs.clay:
            self.resources.clay = MAXINT
        if min(self.robots.obsidian, self.resources.obsidian) >= max_costs.obsidian:
            self.resources.obsidian = MAXINT


    def __hash__(self):
        return hash((self.robots, self.resources))

    def better(self, other):
        res_better = (self.resources.ore >= other.resources.ore) and \
               (self.resources.clay >= other.resources.clay) and \
               (self.resources.obsidian >= other.resources.obsidian) and \
               (self.resources.geode >= other.resources.geode)
        
        rob_better = (self.robots.ore >= other.robots.ore) and \
               (self.robots.clay >= other.robots.clay) and \
               (self.robots.obsidian >= other.robots.obsidian) and \
               (self.robots.geode >= other.robots.geode)

        return (res_better and rob_better)
        
    def max(self, other):
        res = Resources(ore=max(self.resources.ore, other.resources.ore),
                        clay=max(self.resources.clay, other.resources.clay),
                        obsidian=max(self.resources.obsidian, other.resources.obsidian),
                        geode=max(self.resources.geode, other.resources.geode))
    
        rob = Resources(ore=max(self.robots.ore, other.robots.ore),
                        clay=max(self.robots.clay, other.robots.clay),
                        obsidian=max(self.robots.obsidian, other.robots.obsidian),
                        geode=max(self.robots.geode, other.robots.geode))

        return self.__class__(robots=rob, resources=res)



In [408]:
allowed = string.digits + ' '

def make_blueprint(L : List[int]):
    assert len(L) == 7
    ore = Resources(ore=L[1])
    clay = Resources(ore=L[2])
    obsidian = Resources(ore=L[3], clay=L[4])
    geode = Resources(ore=L[5], obsidian=L[6])

    return Blueprint(ore, clay, obsidian, geode)

def parse_line(line):
    parts = ''.join([c for c in line[1:] if c in allowed]).split()
    return [int(p) for p in parts]

def read(prefix='data'):
    data = Path(f'{prefix}/19.txt').read_text().rstrip().split('\n')
    return [make_blueprint(parse_line(l)) for l in data]

In [424]:
def transitions(s, B):
    t = deepcopy(s)
    mcosts = B.max_costs()
    t.mine()
    
    T = []
    names = ['ore', 'clay', 'obsidian', 'geode']
    costs = [B.ore, B.clay, B.obsidian, B.geode]
    bals = s.resources._as_tuple
    sols = 0
    for name, cost, bal in zip(names, costs, bals):
        if s.resources.can_afford(cost):
            sols+=1
            if (bal < MAXINT):
                a = deepcopy(t)
                a.resources.pay(cost)
                a.robots.increment(name)
                a.set_inf(mcosts)
                T.append(a)

    if sols < len(names):
        T.append(t)
        t.set_inf(mcosts)
    
    return set(T)


In [425]:
def prune(states : set):
    kept = set()
    for s in states:
            if not any(k.better(s) for k in kept):
                kept.add(s)
    return kept

In [433]:
def optimise(B : Blueprint, timeleft : int, apply_prune=False, verbose=False):
    states = set([State()])
    for i in range(timeleft):
        new = set()
        for s in states:
            new = new | transitions(s, B)
        if apply_prune:
            new = prune(new)

        states = new
        if verbose:
            print(f'{i+1}/{timeleft}, {len(states)=} max =', max(states, key = lambda s : s.resources.geode))

    
    return max(states, key = lambda s : s.resources.geode)


In [434]:
optimise(read('test')[0], 24, apply_prune=True, verbose=True)

1/24, len(states)=1 max = State(robots=Robots(ore=1, clay=0, obsidian=0, geode=0), resources=Resources(ore=1, clay=0, obsidian=0, geode=0))
2/24, len(states)=1 max = State(robots=Robots(ore=1, clay=0, obsidian=0, geode=0), resources=Resources(ore=2, clay=0, obsidian=0, geode=0))
3/24, len(states)=2 max = State(robots=Robots(ore=1, clay=0, obsidian=0, geode=0), resources=Resources(ore=3, clay=0, obsidian=0, geode=0))
4/24, len(states)=3 max = State(robots=Robots(ore=1, clay=0, obsidian=0, geode=0), resources=Resources(ore=4, clay=0, obsidian=0, geode=0))
5/24, len(states)=5 max = State(robots=Robots(ore=1, clay=0, obsidian=0, geode=0), resources=Resources(ore=5, clay=0, obsidian=0, geode=0))
6/24, len(states)=6 max = State(robots=Robots(ore=1, clay=1, obsidian=0, geode=0), resources=Resources(ore=4, clay=2, obsidian=0, geode=0))
7/24, len(states)=10 max = State(robots=Robots(ore=1, clay=2, obsidian=0, geode=0), resources=Resources(ore=3, clay=6, obsidian=0, geode=0))
8/24, len(states)=1

State(robots=Robots(ore=1, clay=6, obsidian=2, geode=2), resources=Resources(ore=2, clay=47, obsidian=8, geode=9))

In [435]:
optimise(read('test')[1], 24, apply_prune=True, verbose=True)

1/24, len(states)=1 max = State(robots=Robots(ore=1, clay=0, obsidian=0, geode=0), resources=Resources(ore=1, clay=0, obsidian=0, geode=0))
2/24, len(states)=1 max = State(robots=Robots(ore=1, clay=0, obsidian=0, geode=0), resources=Resources(ore=2, clay=0, obsidian=0, geode=0))
3/24, len(states)=2 max = State(robots=Robots(ore=1, clay=0, obsidian=0, geode=0), resources=Resources(ore=3, clay=0, obsidian=0, geode=0))
4/24, len(states)=3 max = State(robots=Robots(ore=1, clay=0, obsidian=0, geode=0), resources=Resources(ore=4, clay=0, obsidian=0, geode=0))
5/24, len(states)=4 max = State(robots=Robots(ore=2, clay=1, obsidian=0, geode=0), resources=Resources(ore=2, clay=0, obsidian=0, geode=0))
6/24, len(states)=7 max = State(robots=Robots(ore=1, clay=1, obsidian=0, geode=0), resources=Resources(ore=3, clay=2, obsidian=0, geode=0))
7/24, len(states)=6 max = State(robots=Robots(ore=4, clay=1, obsidian=0, geode=0), resources=Resources(ore=9223372036854775806, clay=2, obsidian=0, geode=0))
8/

State(robots=Robots(ore=3, clay=7, obsidian=7, geode=4), resources=Resources(ore=9223372036854775806, clay=45, obsidian=9, geode=12))

In [436]:
tot = 0
for idx,b in enumerate(read('test')):
    state = optimise(b, 24, apply_prune=True, verbose=False)
    print(idx+1, state)
    tot += (idx+1)*state.resources.geode

tot

1 State(robots=Robots(ore=1, clay=6, obsidian=2, geode=2), resources=Resources(ore=2, clay=47, obsidian=8, geode=9))
2 State(robots=Robots(ore=3, clay=7, obsidian=7, geode=4), resources=Resources(ore=9223372036854775806, clay=45, obsidian=9, geode=12))


33

In [437]:
tot = 0
for idx,b in enumerate(read()):
    state = optimise(b, 24, apply_prune=True, verbose=False)
    print(idx+1, state)
    tot += (idx+1)*state.resources.geode

tot

1 State(robots=Robots(ore=4, clay=12, obsidian=5, geode=1), resources=Resources(ore=9223372036854775806, clay=48, obsidian=9, geode=1))
2 State(robots=Robots(ore=3, clay=6, obsidian=1, geode=0), resources=Resources(ore=9, clay=41, obsidian=7, geode=0))
3 State(robots=Robots(ore=2, clay=5, obsidian=2, geode=1), resources=Resources(ore=9, clay=31, obsidian=5, geode=1))
4 State(robots=Robots(ore=3, clay=10, obsidian=5, geode=1), resources=Resources(ore=4, clay=23, obsidian=8, geode=1))
5 State(robots=Robots(ore=4, clay=14, obsidian=2, geode=0), resources=Resources(ore=9223372036854775806, clay=104, obsidian=8, geode=0))
6 State(robots=Robots(ore=4, clay=12, obsidian=1, geode=0), resources=Resources(ore=9223372036854775806, clay=69, obsidian=6, geode=0))
7 State(robots=Robots(ore=4, clay=10, obsidian=3, geode=0), resources=Resources(ore=9223372036854775806, clay=47, obsidian=9, geode=0))
8 State(robots=Robots(ore=4, clay=11, obsidian=3, geode=1), resources=Resources(ore=9223372036854775806

2193

In [456]:
tot = 1
for idx,b in enumerate(read()[:3]):
    state = optimise(b, 32, apply_prune=True, verbose=False)
    print(idx+1, state)
    tot *= state.resources.geode

tot

1 State(robots=Robots(ore=4, clay=12, obsidian=10, geode=4), resources=Resources(ore=9223372036854775806, clay=56, obsidian=21, geode=18))
2 State(robots=Robots(ore=4, clay=11, obsidian=7, geode=4), resources=Resources(ore=9223372036854775806, clay=30, obsidian=14, geode=16))
3 State(robots=Robots(ore=4, clay=9, obsidian=6, geode=6), resources=Resources(ore=9223372036854775806, clay=64, obsidian=10, geode=25))


7200